In [ ]:
from dataclasses import dataclass
from typing import Final, overload

import numpy as np
from genx.models.spec_nx import Coords, FootType, Instrument, Layer, Probe, ResType, Sample, Specular, Stack
from numpy.typing import NDArray

XRAY_TUBE: Final[Instrument] = Instrument(
    probe=Probe.xray,
    wavelength=1.54,
    coords=Coords.q,
    I0=1.0,
    Ibkg=1e-10,
    res=0.005,
    restype=ResType.fast_conv, # 분해능 컨볼루션
    footype=FootType.gauss
)

AIR: Final[Layer] = Layer(
    d=0.0,
    f=complex(0, 0),
    dens=0.0,
    sigma=0.0
)

SURFACE_SIO2: Final[Layer] = Layer(
    d=15.0,
    f=complex(14, 0.1),
    dens=0.05,
    sigma=2.0
)

SUBSTRATE_SI: Final[Layer] = Layer(
    d=0.0,
    f=complex(13.5, 0.05),
    dens=0.05,
    sigma=2.0
)

@dataclass
class ParamSet:
    thickness: float
    roughness: float
    sld: float

    def to_numpy(self) -> np.ndarray:
        return np.array([self.thickness, self.roughness, self.sld], dtype=np.float64)

    def __format__(self, format_spec: str) -> str:
        """클래스 전체에 포매팅 명령어 적용 (.3f, .2f 등)"""
        formatted = [
            f"{name}={getattr(self, name):{format_spec}}"
            for name in self.__dataclass_fields__.keys()
        ]
        return f"{self.__class__.__name__}({', '.join(formatted)})"


@overload
def tth2q_wavelen(tth: float, wavelen: float = 1.54) -> float: ...

@overload
def tth2q_wavelen(tth: NDArray[np.float64], wavelen: float = 1.54) -> NDArray[np.float64]: ...

def tth2q_wavelen(tth, wavelen=1.54):
    """
    Convert 2θ (in degrees) and wavelength (in Å) to scattering vector q (in 1/Å).
    """
    th_rad = np.deg2rad(0.5 * tth)
    result = (4 * np.pi / wavelen) * np.sin(th_rad)

    if isinstance(tth, (int, float)):
        return float(result)
    return result

def build_sample(params: list[ParamSet]) -> Sample:
    """genx Sample를 연속 파라미터로부터 생성."""

    layers: list[Layer] = [SURFACE_SIO2]
    for param in params:
        layer = Layer(
            param.thickness,
            f = complex(param.sld, 0),
            dens=1,
            sigma=param.roughness
            )
        layers.append(layer)
    stack = Stack(Layers=layers, Repetitions=1)
    sample = Sample(
        Stacks=[stack],
        Ambient=AIR,
        Substrate=SUBSTRATE_SI
    )

    return sample

def calc_refl(sample: Sample, qs: np.ndarray) -> np.ndarray | None:
    """
    Compute reflectivity including beam footprint correction.

    Parameters
    ----------
    structure : refnx.reflect.Structure
        The interfacial structure.
    q : np.ndarray
        Momentum transfer values (Å⁻¹).

    Returns
    -------
    np.ndarray
        Reflectivity values corrected for beam footprint.
    """
    # 1. 기본 ReflectModel 계산
    reflectivity = Specular(qs, sample, XRAY_TUBE)
    return reflectivity if isinstance(reflectivity, np.ndarray) else None

In [ ]:
wavelen: float = 1.54
tth_min: float = 0.1   # degree
tth_max: float = 13
tth_n: int = 200
tths: np.ndarray = np.linspace(tth_min, tth_max, tth_n)
qs: np.ndarray = tth2q_wavelen(tths, wavelen)

param = ParamSet(20, 1, 2)
sim_refl = calc_refl()

refl = sim_refl
refl = refl / refl.max()
# refl = refl / refl[0]
plt.plot(tths, refl)
plt.yscale("log")